![neurotic logo][logo]

# neurotic

_Curate, visualize, annotate, and share your behavioral ephys data using Python_

Check the [documentation][docs] for help with __neurotic__, including an [API reference guide][api] for the classes and functions used in this notebook.

[api]:  https://neurotic.readthedocs.io/en/latest/api.html
[docs]: https://neurotic.readthedocs.io/en/latest
[logo]: https://raw.githubusercontent.com/jpgill86/neurotic/master/neurotic/gui/icons/img/neurotic-logo-150.png

## Table of Contents

- [1  Setup](#1--Setup)
    - [1.1  Install Packages](#1.1--Install-Packages)
    - [1.2  Import Packages](#1.2--Import-Packages)
- [2  Quick Tour](#2--Quick-Tour)
    - [2.1  Select a Data Set](#2.1--Select-a-Data-Set)
    - [2.2  Load Data and Configure Ephyviewer](#2.2--Load-Data-and-Configure-Ephyviewer)
- [3  API Tutorial](#3--API-Tutorial)
    - [3.1  Minimal Working Examples](#3.1--Minimal-Working-Examples)
    - [3.2  Adding Complexity](#3.2--Adding-Complexity)
    - [3.3  Storing Metadata in a File](#3.3--Storing-Metadata-in-a-File)
    - [3.4  Reading Metadata from a File](#3.4--Reading-Metadata-from-a-File)
    - [3.5  Downloading Files](#3.5--Downloading-Files)
    - [3.6  Loading Data Sets](#3.6--Loading-Data-Sets)
    - [3.7  Configuring the GUI](#3.7--Configuring-the-GUI)
    - [3.8  Jupyter Notebook Widgets](#3.8--Jupyter-Notebook-Widgets)

## 1  Setup

### 1.1  Install Packages

Run this cell to determine whether **neurotic** is installed:

In [ ]:
!pip show neurotic

If **neurotic** is not installed, install it now:

In [ ]:
# install the latest release version
# !pip install -U neurotic

# alternatively, install the latest development version
# !pip install -U git+https://github.com/jpgill86/neurotic.git

### 1.2  Import Packages

Run the cell below to import packages needed for the examples.

In [ ]:
import neo
import neurotic

## 2  Quick Tour

### 2.1  Select a Data Set

The next cell will download `metadata.yml`, a file containing information about available data sets and how they should be loaded, prepared, and displayed, including:
* locations of data files
* video synchronization parameters
* plot settings
* filter parameters
* amplitude windows for spike discrimination

In [ ]:
# download the metadata file
remote_metadata = 'https://raw.githubusercontent.com/jpgill86/neurotic/master/neurotic/example/metadata.yml'
neurotic.download(remote_metadata, './metadata.yml', overwrite_existing=False)

The `MetadataSelectorWidget` allows you to view the names and descriptions of data sets listed in `metadata.yml` and to select the one you would like to work with. Each data set may contain multiple files, such as an electrophysiology data file and a video file. Here is the key for icons found at the left of each entry:<br/>
&nbsp;&nbsp; ◆ &nbsp;&nbsp; all files can be found locally and none need to be downloaded<br/>
&nbsp;&nbsp; ⬖ &nbsp;&nbsp; some files can be found locally and others cannot<br/>
&nbsp;&nbsp; ◇ &nbsp;&nbsp; no files can be found locally and all need to be downloaded<br/>
&nbsp;&nbsp;&nbsp; ! &nbsp;&nbsp;&nbsp; the `video_offset` parameter is not set, which can cause out-of-sync video playback

__This example contains only one data set, so this part is easy!__

In [ ]:
# choose a data set to load
metadata = neurotic.MetadataSelectorWidget(file='./metadata.yml')
display(metadata)

The next cell will download all of the data files associated with the selected data set. Unless `overwrite_existing=True`, a file will only be downloaded if a copy isn't found locally to avoid overwritting local changes.

In [ ]:
# download the data files
metadata.download_all_data_files(overwrite_existing=False)

### 2.2  Load Data and Configure Ephyviewer

The next cell will use the settings contained in `metadata.yml` for the selected data set to do the following:
* read the electrophysiology data file
* apply filters to signals (`lazy=False` only)
* read annotations contained in CSV files
* run a simple spike detection algorithm using amplitude windows (`lazy=False` only)
* import spikes previously sorted by tridesclous
* calculate firing rates (`lazy=False` only)
* calculate rectified area under the curve (RAUC) time series for each signal (`lazy=False` only)

When this is complete, a configuration widget will display that allows you to control which of the modular data viewers you would like the application to show. Click the "Launch" button to start up the application.

__Each time a new data set is selected using the `MetadataSelectorWidget`, you must rerun this cell.__

In [ ]:
# this cell must be rerun each time a new data set is selected above

# lazy loading takes advantage of Neo's high performance RawIO classes,
# which can read portions of files "on demand" to get just the data that
# is needed for plotting the currently visible time span
# - this decreases loading time and consumes much less memory, especially
#   for large files
# - to take advantage of these benefits, signal filtering, amplitude
#   window spike discrimination, firing rate, and RAUC computation must
#   be disabled
# - spike markers on signals are currently incompatible with lazy loading
lazy = False

blk = neurotic.load_dataset(metadata, lazy=lazy)

ephyviewer_config = neurotic.EphyviewerConfiguratorWidget(metadata, blk, lazy)
ephyviewer_config.show_all()
display(ephyviewer_config)

Executing the next cell is equivalent to pressing the "Launch" button and is provided for the convenience of running all cells in the notebook at once to start the application.

In [ ]:
# using the buttons above, select the viewers you'd like to see,
# then either click "Launch" or run this cell
ephyviewer_config.launch_ephyviewer()

After launching, you should see something like the following:

![example screenshot][screenshot]

[screenshot]: https://raw.githubusercontent.com/jpgill86/neurotic/master/docs/_static/example-screenshot.png

## 3  API Tutorial

This section provides an introduction to **neurotic**'s API.

Before getting started, run this cell to download a small data set needed for the examples below.

In [ ]:
metadata = neurotic.MetadataSelector(file='./metadata.yml')
metadata.select('example dataset')
metadata.download_all_data_files(overwrite_existing=False)

### 3.1  Minimal Working Examples

**neurotic** provides a function called [``neurotic.quick_launch``][quick_launch], which is the easiest way to get started with the API.

[quick_launch]: https://neurotic.readthedocs.io/en/latest/api/scripts.html#neurotic.scripts.quick_launch

[Neo][Neo] users can call ``quick_launch`` with a [``neo.Block``][neo.Block] object that has already been loaded into memory to visualize data using the [ephyviewer][ephyviewer] library. **neurotic** handles the configuration of the user interface and, in the simplest case, just passes the ``neo.Block`` to the appropriate viewers.

For example, try running the cell below. A window will open (possibly behind your browser window) that displays the contents of a randomly generated ``neo.Block``. You may wish to rescale the signals, which can be done by pressing the "Auto scale" button. You must close the window before moving on to the next example.

[ephyviewer]:   https://ephyviewer.readthedocs.io
[Neo]:          https://neo.readthedocs.io
[neo.Block]:    https://neo.readthedocs.io/en/stable/api_reference.html#neo.core.Block

In [ ]:
# randomly generate a Neo Block containing signals, spike trains, epochs, and events
blk = neo.test.generate_datasets.generate_one_simple_block(nb_segment=1, supported_objects=neo.objectlist)

# start a GUI that plots the contents of the Block
neurotic.quick_launch(blk=blk)

Where **neurotic** really shines is if you let it handle data loading and processing for you. Instead of (or in addition to) calling ``quick_launch`` with a ready-to-go ``neo.Block``, you may call it with configuration instructions, called "metadata". Metadata may be as simple as a dictionary containing only the path to an electrophysiology data file.

In the example below, we define a simple metadata dictionary containing the path to an example ephys file in the AxoGraph format. When ``quick_launch`` executes, it reads the file into a ``neo.Block`` and then displays it in the user interface.

In [ ]:
metadata = {'data_file': 'example-data/data.axgx'}
neurotic.quick_launch(metadata)

### 3.2  Adding Complexity

By adding to the ``metadata`` dictionary, we can get **neurotic** to do much more with the data. Documentation about configuring metadata can be found here: [Configuring Metadata][metadata].

For example, we can specify [amplitude thresholds][spike detection] so that spikes can be detected when the data is loaded. After running the cell below, notice that peaks are marked on the "BN2" channel, and a raster plot of the spike train appears below the signals.

This requires that we disable "lazy loading" of data files. When enabled (default), "lazy loading" or "fast loading" is used for speeding up load times and minimizing memory usage.

[metadata]:        https://neurotic.readthedocs.io/en/latest/metadata.html
[spike detection]: https://neurotic.readthedocs.io/en/latest/metadata.html#amplitude-discriminators

In [ ]:
metadata = {
    'data_file': 'example-data/data.axgx',

    'amplitude_discriminators': [
        {
            'name':      'B3 neuron',
            'channel':   'BN2',
            'units':     'uV',
            'amplitude': [50, 150],
        },
    ],
}

# lazy=False is required for amplitude_discriminators
neurotic.quick_launch(metadata, lazy=False)

Next, let's add a low-pass [filter][filters] to smooth out the force transducer signal. Notice that the high-frequency noise in the force signal has been reduced.

[filters]: https://neurotic.readthedocs.io/en/latest/metadata.html#filters

In [ ]:
metadata = {
    'data_file': 'example-data/data.axgx',
    
    'amplitude_discriminators': [
        {
            'name':      'B3 neuron',
            'channel':   'BN2',
            'units':     'uV',
            'amplitude': [50, 150],
        },
    ],
    
    'filters': [
        {
            'channel': 'Force',
            'lowpass': 50 # Hz
        },
    ],
}

# lazy=False is required for amplitude_discriminators and filters
neurotic.quick_launch(metadata, lazy=False)

The behavior of the animal that generated this example data (a sea slug feeding on seaweed) was recorded during the experiment. With a couple more additions to the metadata, we can load the video too and synchronize it with the signals.

In [ ]:
metadata = {
    'data_file': 'example-data/data.axgx',

    'amplitude_discriminators': [
        {
            'name':      'B3 neuron',
            'channel':   'BN2',
            'units':     'uV',
            'amplitude': [50, 150],
        },
    ],

    'filters': [
        {
            'channel': 'Force',
            'lowpass': 50 # Hz
        },
    ],

    'video_file': 'example-data/video.mp4',
    'video_offset': 640.3, # seconds <--- used for aligning video and data
}

# lazy=False is required for amplitude_discriminators and filters
neurotic.quick_launch(metadata, lazy=False)

### 3.3  Storing Metadata in a File

The examples above demonstrate how to use **neurotic** to get something working quickly. However, you may wish to maintain metadata for multiple data sets in a more convenient form. For this, **neurotic** supports reading metadata from [YAML][YAML] files. For example, the metadata shown above could be stored like this in a plain text file (typically with the file extension ``*.yml`` or ``*.yaml``):

``` yaml
example dataset:
    data_file: example-data/data.axgx

    amplitude_discriminators:
        - name:      B3 neuron
          channel:   BN2
          units:     uV
          amplitude: [50, 150]

    filters:
        - channel: Force
          lowpass: 50 # Hz

    video_file: example-data/video.mp4
    video_offset: 640.3 # seconds
```

[YAML]: https://en.wikipedia.org/wiki/YAML

Multiple metadata sets can be included in a single YAML file. These could correspond to different data sets (with different files), or to different configurations using the same data set. The following example shows how multiple configurations for the example data could be stored in a single YAML file. Each must have a unique key (e.g., ``minimal example``, ``fancy example``) and can be given a ``description`` to distinguish them.

``` yaml
minimal example:
    description: 'Load just the raw data'

    data_file: example-data/data.axgx

fancy example:
    description: 'Load data and video, apply filters, detect spikes'

    data_file: example-data/data.axgx

    amplitude_discriminators:
        - name:      B3 neuron
          channel:   BN2
          units:     uV
          amplitude: [50, 150]

    filters:
        - channel: Force
          lowpass: 50 # Hz

    video_file: example-data/video.mp4
    video_offset: 640.3 # seconds
```

To see how this can be used, first run the cell below to create a new file called ``demo.yml`` containing the YAML-formatted metadata shown above.

In [ ]:
file_contents = \
"""
minimal example:
    description: 'Load just the raw data'

    data_file: example-data/data.axgx

fancy example:
    description: 'Load data and video, apply filters, detect spikes'

    data_file: example-data/data.axgx

    amplitude_discriminators:
        - name:      B3 neuron
          channel:   BN2
          units:     uV
          amplitude: [50, 150]

    filters:
        - channel: Force
          lowpass: 50 # Hz

    video_file: example-data/video.mp4
    video_offset: 640.3 # seconds
"""

with open('demo.yml', 'w') as f:
    f.write(file_contents)

### 3.4  Reading Metadata from a File

To read and manage metadata in a YAML file, we can create a [``neurotic.MetadataSelector``][MetadataSelector] object:

[MetadataSelector]: https://neurotic.readthedocs.io/en/latest/api/metadata.html#neurotic.datasets.metadata.MetadataSelector

In [ ]:
metadata = neurotic.MetadataSelector(file='demo.yml')

When a YAML file is read using the ``MetadataSelector``, default values are filled in automatically for missing parameters. The contents of the loaded file can be displayed using the ``all_metadata`` attribute.

In [ ]:
metadata.all_metadata

After a ``MetadataSelector`` has loaded a file, you must select which entry in the file you wish to use. This is done using the ``select()`` method. You may use ``keys`` to list the available entry names.

In [ ]:
metadata.keys

Let's select the "``fance example``", which includes video, an analog signal filter, and spike detection.

In [ ]:
metadata.select('fancy example')

Now, through the magic of the ``MetadataSelector`` class, the ``metadata`` object can be treated like the dictionaries created earlier for the simpler examples. For instance, parameters for the selected configuration can be extracted like this:

In [ ]:
metadata['data_file']

The ``MetadataSelector`` instance can also be passed to ``quick_launch``:

In [ ]:
neurotic.quick_launch(metadata, lazy=False)

### 3.5  Downloading Files

If data files are made available for download from the internet, metadata can be configured so that ``MetadataSelector`` will be able to download them on demand using the ``download`` and ``download_all_data_files`` methods. This feature was used at the start of this tutorial to retrieve the files needed to run these examples.

For more information about configuring metadata for downloads, [see the documentation][remote data].

[remote data]: https://neurotic.readthedocs.io/en/latest/metadata.html#remote-data-available-for-download

### 3.6  Loading Data Sets

``quick_launch`` is a convenience function that wraps two fundamental procedures into one:

1. Load and process the data according to the specifications given in the selected metadata, resulting in a [``neo.Block``][neo.Block].
2. Configure and launch the interactive user interface, displaying the data in the ``neo.Block`` (and possibly other data such as video).

We will explore the first of these procedures in more detail now.

[neo.Block]:    https://neo.readthedocs.io/en/stable/api_reference.html#neo.core.Block

The [``neurotic.load_dataset``][load_dataset] function is used by ``quick_launch``---and can be used by you directly---to read data from files and apply procedures like signal filtering and spike detection. ``load_dataset`` uses metadata information passed to it to locate files and determine which, if any, data processing steps to take. It returns a [``neo.Block``][neo.Block] object for representing the loaded and processed neuroscience data.

[load_dataset]: https://neurotic.readthedocs.io/en/latest/api/data.html#neurotic.datasets.data.load_dataset
[neo.Block]:    https://neo.readthedocs.io/en/stable/api_reference.html#neo.core.Block

For example, we can use ``load_dataset`` with the "``fancy example``" specifications to load the example data file, apply a low-pass filter to the "Force" channel, run a spike detection algorithm on the "BN2" channel, and then inspect the results.

In [ ]:
metadata = neurotic.MetadataSelector(file='demo.yml')
metadata.select('fancy example')
blk = neurotic.load_dataset(metadata, lazy=False)

In [ ]:
# show the entire neo.Block
display(blk)

In [ ]:
# show the times at which spikes were detected
display(blk.segments[0].spiketrains[0].times)

### 3.7  Configuring the GUI

To launch the graphical user interface, which is built using the [ephyviewer][ephyviewer] library, ``quick_launch`` uses the [``neurotic.EphyviewerConfigurator``][EphyviewerConfigurator] class. By using it directly in your code instead of ``quick_launch``, you can obtain greater control over which data types are displayed and over the appearance of the interface.

[ephyviewer]:             https://ephyviewer.readthedocs.io
[EphyviewerConfigurator]: https://neurotic.readthedocs.io/en/latest/api/config.html#neurotic.gui.config.EphyviewerConfigurator

For instance, you may wish to visualize the "``fancy example``" data set and to display spikes as points plotted on the signal, but you might want to hide the raster plot of the spike train because it contains redundant information. In the cell below, ``EphyviewerConfigurator`` is used to control which data viewers are hidden. It is also used to enlarge the interface text.

In [ ]:
metadata = neurotic.MetadataSelector(file='demo.yml')
metadata.select('fancy example')
blk = neurotic.load_dataset(metadata, lazy=False)
ephyviewer_config = neurotic.EphyviewerConfigurator(metadata, blk, lazy=False)

# do not show raster plot of spike train
ephyviewer_config.hide('spike_trains')

# launch the GUI with large fonts
ephyviewer_config.launch_ephyviewer(ui_scale='large')

### 3.8  Jupyter Notebook Widgets

When working in Jupyter notebooks, you may use [``neurotic.MetadataSelectorWidget``][MetadataSelectorWidget] in place of ``MetadataSelector`` to read YAML files and display an interactive list.

[MetadataSelectorWidget]: https://neurotic.readthedocs.io/en/latest/api/notebook.html#neurotic.gui.notebook.MetadataSelectorWidget

In [ ]:
metadata = neurotic.MetadataSelectorWidget(file='demo.yml')
display(metadata)

The `MetadataSelectorWidget` allows you to view the keys and descriptions of data sets listed in the YAML file and to select the one you would like to work with. The "Reload" button at the bottom of the widget reloads the YAML file's contents.

Each data set may contain multiple files, such as an electrophysiology data file and a video file. Here is the key for icons found at the left of each entry:<br/>
&nbsp;&nbsp; ◆ &nbsp;&nbsp; all files can be found locally<br/>
&nbsp;&nbsp; ⬖ &nbsp;&nbsp; some files can be found locally and others cannot (these may need to be downloaded)<br/>
&nbsp;&nbsp; ◇ &nbsp;&nbsp; no files can be found locally (all may need to be downloaded)<br/>
&nbsp;&nbsp;&nbsp; ! &nbsp;&nbsp;&nbsp; the `video_offset` parameter is not set, which can cause out-of-sync video playback

By clicking on an entry in the widget above, you select which metadata set is used when the `MetadataSelectorWidget` instance is passed to other **neurotic** functions, like ``quick_launch``. Try changing the selection above and running the cell below. Notice that the entry name appears in the window title of the app.

In [ ]:
neurotic.quick_launch(metadata, lazy=False)

In Jupyter notebooks, you may also use [``neurotic.EphyviewerConfiguratorWidget``][EphyviewerConfiguratorWidget] instead of ``EphyviewerConfigurator`` for controlling which viewers are visible and launching the interface.

Note, however, that the ``EphyviewerConfiguratorWidget`` must be reconstructed if the metadata selection changes, and ``load_dataset`` must also be run first.

[EphyviewerConfiguratorWidget]: https://neurotic.readthedocs.io/en/latest/api/notebook.html#neurotic.gui.notebook.EphyviewerConfiguratorWidget

In [ ]:
metadata = neurotic.MetadataSelector(file='demo.yml')
metadata.select('fancy example')
blk = neurotic.load_dataset(metadata, lazy=False)
ephyviewer_config = neurotic.EphyviewerConfiguratorWidget(metadata, blk, lazy=False)
display(ephyviewer_config)